Mittlere ACF (0–50 Lags)


Langsames Abklingen
Die ACF startet bei 1.0 (Lag 0) und fällt nur langsam ab – schon bei Lag 50 liegt sie noch bei rund 0.75. Das spricht für eine sehr starke Persistenz, d.h. Verbräuche bleiben auch über mehrere Stunden hinweg hoch korreliert.

Bump um Lag 24
Zwischen Lag 20–28 siehst du einen kleinen „Höcker“ (ACF steigt von ~0.78 wieder auf ~0.80). Das ist genau der tägliche Saison-Effekt (24 h), der deine Zeitreihen zusätzlich stützt.

Fehlende deutliche Peaks bei Lag 7×24 oder 168
Da du nur bis Lag 50 geplottet hast, siehst du den Wochen-Peak (168 h) hier noch nicht. Würdest du bis Lag 168 gehen, käme dort ein weiterer, kleiner Bump.

Fazit aus der ACF:

Starkes Grundrauschen mit sehr langsamer Abklingrate

Eindeutige tägliche Saisonkorrelation bei Lag 24

Mittlere PACF (0–50 Lags)


Spitze bei Lag 1
Der PACF-Wert bei Lag 1 ist praktisch 1.0 → deutet auf einen AR(1)-Anteil hin.

Negativer Spike bei Lag 2
Direkt bei Lag 2 geht der PACF kurz unter 0 (rund –0.2) → ein Hinweis darauf, dass auch ein MA(1) – oder kombiniertes ARMA(1,1) – Modell passen könnte.

Kleine Hügel bei Lag 24–25
Im Bereich um Lag 24–25 tauchen im PACF noch einmal kleine positive Ausreißer auf, was auf einen saisonalen AR-Term bei einer Periodenlänge von 24 h hindeutet.

Schnelles Abklingen danach
Ab Lag 30 ist der PACF für die meisten Lags nahe null → hier ist kein weiterführender rein nicht-saisonaler AR-Order mehr nötig.

Fazit aus der PACF:

Ein nicht‐saisonaler AR(1) ist sinnvoll.

Ein MA(1) (bei Lag 2 negativer Effekt) kann helfen, das erste Abklingen zu modellieren.

Ein saisonaler AR(1) bei Lag 24 (für den Tageszyklus) ist gerechtfertigt.

Konsequenz für dein Basis-SARIMAX
Nicht‐saisonaler Teil: wahrscheinliche Kandidaten sind
(

,

,

)
=
(
1
,
0
,
1
)
(p,d,q)=(1,0,1) (da ADF-Test + ACF/PACF stationär ohne Differenzierung).

Saisonaler Teil (m=24): ein

=
1
P=1 aus dem PACF-Hügel bei Lag 24, und ggfs.

=
0
Q=0 (keine klare Cut-Off in der ACF).

Weitere Saisonen (168 h / 8760 h) behandelst du idealerweise exogen (Fourier oder Dummies) oder per MSTL-Features.

Damit hättest du für dein Basis-SARIMAX in etwa:

python
Copy
Edit
seasonal_order=(1,0,0,24)    # P=1, D=0, Q=0, m=24
order=(1,0,1)                # p=1, d=0, q=1
plus deine exogenen Features für Wochen‐ und Jahreseffekte. So schlagen deine ACF/PACF-Erkenntnisse direkt in Parameterempfehlungen um.


In [3]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from utils import load_series_dfs
import pickle

In [6]:
series_dfs = load_series_dfs(data_dir=r"C:\Users\johan\Documents\FH_Master\data", filename="final_learning_dfs.pkl")
series_dfs['FL_00024702.PLAN.MengeHH'].columns

Index(['consumption', 'hour', 'weekday', 'month', 'is_weekend', 'w_tl', 'w_rf',
       'w_ff', 'w_ffx', 'w_cglo', 'w_so_h', 'w_rr', 'w_rrm', 'w_tb10',
       'w_tb20', 'CEGH_WAP', 'THE_WAP'],
      dtype='object')

In [5]:
with open('../data/mstl_results.pkl', 'rb') as f:
    mstl_results = pickle.load(f)

In [11]:
# --- 1) Panel aufbauen wie gehabt, aber ohne series_key ---
rows = []
for key, df in series_dfs.items():
    tmp = df[['consumption',
              'hour','weekday','month','is_weekend',
              'w_tl','w_rf','w_ff','w_ffx','w_cglo','w_so_h',
              'w_rr','w_rrm','w_tb10','w_tb20',
              'CEGH_WAP','THE_WAP']]  # Deine Wetter- und Preis-Features
    m = mstl_results[key].seasonal.rename(columns={
        'seasonal_24':'s24','seasonal_168':'s168','seasonal_8760':'s8760'
    })
    tmp = tmp.join(m)
    tmp['series_key'] = key
    rows.append(tmp)

panel = pd.concat(rows).reset_index()

In [15]:
panel.columns

Index(['von  | von', 'consumption', 'hour', 'weekday', 'month', 'is_weekend',
       'w_tl', 'w_rf', 'w_ff', 'w_ffx', 'w_cglo', 'w_so_h', 'w_rr', 'w_rrm',
       'w_tb10', 'w_tb20', 'CEGH_WAP', 'THE_WAP', 's24', 's168', 's8760',
       'series_key'],
      dtype='object')

In [16]:
panel_exog = panel.drop(columns=['consumption', 'von  | von'])

X = pd.get_dummies(
    panel_exog,
    columns=['series_key'],
    drop_first=True
)

# 3) Sicherstellen, dass alles float ist:
X = X.astype('float64')

# 4) Endogen definieren
y = panel['consumption'].astype('float64')


In [18]:


# 3) Globales SARIMAX auf dem Panel
model = SARIMAX(
    endog=y,
    exog=X,
    order=(1,0,1),
    seasonal_order=(1,0,0,24),
    enforce_stationarity=False,
    enforce_invertibility=False
)
res = model.fit(disp=False)
print(res.summary())

MemoryError: Unable to allocate 8.81 GiB for an array with shape (25, 25, 1891998) and data type float64

In [9]:
print(panel.dtypes)

von  | von     datetime64[ns]
consumption           float64
hour                    int32
weekday                 int32
month                   int32
is_weekend              int64
w_tl                  float64
w_rf                  float64
w_ff                  float64
w_ffx                 float64
w_cglo                float64
w_so_h                float64
w_rr                  float64
w_rrm                 float64
w_tb10                float64
w_tb20                float64
CEGH_WAP              float64
THE_WAP               float64
s24                   float64
s168                  float64
s8760                 float64
series_key             object
dtype: object


In [10]:
print(X.dtypes.value_counts())

bool       50
float64    15
int32       3
int64       1
Name: count, dtype: int64
